In [62]:
import requests

In [63]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
data = requests.get(standings_url)

In [64]:
from bs4 import BeautifulSoup # parsing html
soup = BeautifulSoup(data.text)

In [65]:
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/206d90db/Barcelona-Stats',
 'https://fbref.com/en/squads/53a2f082/Real-Madrid-Stats',
 'https://fbref.com/en/squads/db3b9613/Atletico-Madrid-Stats',
 'https://fbref.com/en/squads/2b390eca/Athletic-Club-Stats',
 'https://fbref.com/en/squads/2a8183b3/Villarreal-Stats',
 'https://fbref.com/en/squads/fc536746/Real-Betis-Stats',
 'https://fbref.com/en/squads/2aa12281/Mallorca-Stats',
 'https://fbref.com/en/squads/f25da7fb/Celta-Vigo-Stats',
 'https://fbref.com/en/squads/98e8af82/Rayo-Vallecano-Stats',
 'https://fbref.com/en/squads/ad2be733/Sevilla-Stats',
 'https://fbref.com/en/squads/7848bd64/Getafe-Stats',
 'https://fbref.com/en/squads/e31d1cd9/Real-Sociedad-Stats',
 'https://fbref.com/en/squads/9024a00a/Girona-Stats',
 'https://fbref.com/en/squads/03c57e2b/Osasuna-Stats',
 'https://fbref.com/en/squads/a8661628/Espanyol-Stats',
 'https://fbref.com/en/squads/dcc91a7b/Valencia-Stats',
 'https://fbref.com/en/squads/8d6fd021/Alaves-Stats',
 'https://fbref.com/en/

In [66]:
team_url = team_urls[0]
data = requests.get(team_url)


In [67]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")
matches[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,Valencia,3.2,1.0,63.0,46673.0,Marc-André ter Stegen,4-2-3-1,4-4-2,NaN,Match Report,NaN
1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,Athletic Club,1.8,1.0,64.0,46448.0,Marc-André ter Stegen,4-2-3-1,4-2-3-1,NaN,Match Report,NaN
2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,Rayo Vallecano,1.4,0.4,64.0,14031.0,Marc-André ter Stegen,4-2-3-1,4-4-2,NaN,Match Report,NaN
3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,Valladolid,4.7,0.5,70.0,44359.0,Marc-André ter Stegen,4-3-3,4-1-4-1,Isidro Díaz de Mera,Match Report,NaN
4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,Girona,1.9,1.3,55.0,13891.0,Marc-André ter Stegen,4-2-3-1,4-2-3-1,Alejandro Muñíz,Match Report,NaN


In [69]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
links

['/en/squads/206d90db/2024-2025/matchlogs/all_comps/shooting/Barcelona-Match-Logs-All-Competitions',
 '/en/squads/206d90db/2024-2025/matchlogs/all_comps/shooting/Barcelona-Match-Logs-All-Competitions',
 '/en/squads/206d90db/2024-2025/matchlogs/all_comps/shooting/Barcelona-Match-Logs-All-Competitions',
 '/en/squads/206d90db/2024-2025/matchlogs/all_comps/shooting/Barcelona-Match-Logs-All-Competitions']

In [70]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]

In [75]:
shooting.columns = shooting.columns.droplevel()
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,Valencia,...,18.6,1.0,1,1,3.2,2.4,0.14,-1.2,-1.4,Match Report
1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,Athletic Club,...,16.6,0.0,0,0,1.8,1.8,0.14,0.2,0.2,Match Report
2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,Rayo Vallecano,...,19.3,1.0,0,0,1.4,1.4,0.06,0.6,0.6,Match Report
3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,Valladolid,...,13.7,1.0,0,0,4.7,4.7,0.21,2.3,2.3,Match Report
4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,Girona,...,19.1,0.0,0,0,1.9,1.9,0.09,2.1,2.1,Match Report


In [76]:
team_data = matches[0].merge(shooting[['Date', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on="Date")
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Opp Formation,Referee,Match Report,Notes,SoT,Dist,FK,PK,PKatt
0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,Valencia,...,4-2-3-1,4-4-2,NaN,Match Report,NaN,5,18.6,1.0,1,1
1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,Athletic Club,...,4-2-3-1,4-2-3-1,NaN,Match Report,NaN,5,16.6,0.0,0,0
2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,Rayo Vallecano,...,4-2-3-1,4-4-2,NaN,Match Report,NaN,5,19.3,1.0,0,0
3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,Valladolid,...,4-3-3,4-1-4-1,Isidro Díaz de Mera,Match Report,NaN,11,13.7,1.0,0,0
4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,Girona,...,4-2-3-1,4-2-3-1,Alejandro Muñíz,Match Report,NaN,9,19.1,0.0,0,0
